# Grain-level analysis

Notes

## Import modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import transform as tf
from skimage import io

import hrdic
import ebsd

import copy

%matplotlib osx

## Load DIC data

In [4]:
#Load in DIC map
DicFilePath = "./example_data_AH/"
DicMap = hrdic.Map(DicFilePath, "hrdicData.txt")

Loaded DaVis 8.1.5 data (dimensions: 1860 x 1628 pixels, sub-window size: 8 x 8 pixels)


## Load EBSD data

In [5]:
mapFiles = ["./example_data_AH/ebsdData"]

maps = []

for i, mapFile in enumerate(mapFiles):
    maps.append(ebsd.Map(mapFile, "cubic"))
    maps[i].loadData(mapFile, "cubic")
    maps[i].binData = maps[i].binData[::-1]  #rotate the map 180 degrees
    maps[i].buildQuatArray()
    maps[i].findBoundaries(boundDef = 6)
    maps[i].findGrains(minGrainSize=10)
    maps[i].calcGrainMisOri(calcAxis = True)

Loaded EBSD data (dimensions: 612 x 465 pixels, step size: 0.5 um)
Error with cpr file. Number of phases mismatch.
Loaded EBSD data (dimensions: 612 x 465 pixels, step size: 0.5 um)


## Link DIC and EBSD data

In [9]:
# load the disMap crop parameters
DicMap.cropDists = np.loadtxt("./example_data_AH/dicMapCropDists.txt")

# set the dic map crop parameters
DicMap.cropDists_temp=[]
for p,q in DicMap.cropDists:
    p,q = int(p),int(q)
    DicMap.cropDists_temp.append([p,q])
DicMap.cropDists_temp=np.array(DicMap.cropDists_temp)
DicMap.cropDists=copy.deepcopy(DicMap.cropDists_temp)

# define the crop
DicMap.setCrop(xMin=DicMap.cropDists[0,0], xMax=DicMap.cropDists[0,1], yMin=DicMap.cropDists[1,0], yMax=DicMap.cropDists[1,1])

# load and define the homogPoints
DicMap.homogPoints = np.loadtxt("./example_data_AH/dicMapHomogPoints.txt")
maps[0].homogPoints = np.loadtxt("./example_data_AH/ebsdMapHomogPoints.txt")

# link the dic and ebsd data
DicMap.linkEbsdMap(maps[0])

# find the grains
DicMap.findGrains(minGrainSize=1434)

### Test the link has worked

In [10]:
DicMap.locateGrainID()

Grain ID: 55


In [11]:
# test the link has worked
grainNum = DicMap.currGrainId
DicMap.grainList[grainNum].plotMaxShear()
maps[0].grainList[DicMap.ebsdGrainIds[grainNum]].plotMisOri()

## Plot correlation between DIC and EBSD data

In [13]:
# Define the DicMap pixel resolution in order to obtain grain size information
DicMap.pixelResolution = 0.117 # um

In [14]:
# plot data correlation function
def plotCorrelation(dicSelf,ebsdSelf,grainDiameter=False):
    plt.figure()
    for i,grain in enumerate(dicSelf):
        x = np.nanmean(grain.maxShearList)
        y = np.nanmean((np.arccos(ebsdSelf.grainList[dicSelf.ebsdGrainIds[i]].misOriList))*180/np.pi)
        if grainDiameter:
            area = len(grain.maxShearList)
            diameter = ((area/np.pi)**0.5)*2*dicSelf.pixelResolution # um
            plt.scatter(x,y,c=diameter,s=diameter,vmin=20,vmax=40,cmap="viridis")
        else:
            plt.scatter(x,y,c="grey")
    if grainDiameter:    
        plt.colorbar(label="Grain diameter [$\\mu$ m]", shrink=0.8)
    plt.xlabel("HRDIC strain, $\epsilon_{eff}$ [ ]")
    plt.ylabel("EBSD misorientation, $\phi$ [ $^o$ ]")

In [15]:
# plot the correlation
plotCorrelation(DicMap,maps[0],grainDiameter=True)

## Find the strain heterogeneity within the grain

In [36]:
def findEffStrainLocalisationGradient(self,plotData = False):
    
    meanEffStrain = np.nanmean(self.maxShearList)
    hist = np.histogram(np.asarray(self.maxShearList)/meanEffStrain,bins=100)
    x = 0.5*(hist[1][1:]+hist[1][:-1])
    y = hist[0]/np.nansum(hist[0])
    
    # find maximum
    maxValIdx=np.argmax(y)
    xAtMaxVal=x[maxValIdx]
    yAtMaxVal=y[maxValIdx]
    
    # find minimum after maximum value
    for xVal,yVal in zip(x[maxValIdx:],y[maxValIdx:]):
        if yVal < 0.005:
            break
    xAtMinVal=xVal
    yAtMinVal=yVal
    
    # plt the data
    if plotData:

        plt.figure()
        plt.plot(x,y,"-")
        plt.xscale("log")
        plt.plot(xAtMaxVal,yAtMaxVal,"o")
        plt.plot(xAtMinVal,yAtMinVal,"o")
        print("x at max y = " + str(xAtMaxVal))
        print("x at min y = " + str(xAtMinVal))
        
    # find the gradient
    effStrainLocGradient=(yAtMaxVal-yAtMinVal)/(xAtMaxVal-xAtMinVal)
    
    self.effStrainLocGradient = effStrainLocGradient

hrdic.Grain.findEffStrainLocalisationGradient = findEffStrainLocalisationGradient

In [37]:
DicMap.grainList[32].findEffStrainLocalisationGradient(plotData=False)

### Do for all grains and plot

In [39]:
for grain in DicMap.grainList:
    grain.findEffStrainLocalisationGradient(plotData=False)

In [43]:
np.nanmean((np.arccos(maps[0].grainList[DicMap.ebsdGrainIds[32]].misOriList))*180/np.pi)

0.47548536889262372

In [49]:
plt.figure()

for i,grain in enumerate(DicMap.grainList):
    
    i_ebsd = DicMap.ebsdGrainIds[i]
    
    meanEffStrain = np.nanmean(grain.maxShearList)
    meanMisOri = np.nanmean((np.arccos(maps[0].grainList[DicMap.ebsdGrainIds[i]].misOriList))*180/np.pi)
    effStrainLocGrad = grain.effStrainLocGradient
        
    plt.scatter(meanEffStrain,effStrainLocGrad,c=meanMisOri,vmin=0,vmax=1,cmap="viridis")
    
plt.colorbar(shrink=0.8,label="EBSD GROD Mean [ $^o$ ]")
plt.xlabel("HRDIC strain, $\\epsilon_{eff}$ [  ]")
plt.ylabel("$\\epsilon_{eff}$ distribution gradient [  ]")

## Deformation at boundaries relative to the grain centroid: EBSD misOri

#### Calculate GB proxigram

In [50]:
def calcProxiGram(self):
    indexBoundaries = []
    for index, value in np.ndenumerate(self.boundaries):
        if value == -1:
            indexBoundaries.append(index)

    proxBoundaries = np.copy(self.boundaries[:,:])

    numTrials = 1000
    trialDistances = np.ones((numTrials+1, proxBoundaries.shape[0], proxBoundaries.shape[1]))*1000

    # array of x and y coordinate of each pixel in the map
    coords = np.zeros((2, proxBoundaries.shape[0], proxBoundaries.shape[1]))
    for index, value in np.ndenumerate(proxBoundaries):
        coords[0, index[0], index[1]] = index[0]
        coords[1, index[0], index[1]] = index[1]

    # loop over each boundary point (p) and calcuale distance from p to all points in the map
    # once numTrails have been made the minimum of them will be found and stored
    j=1
    for indexBoundary in indexBoundaries:    
        trialDistances[j] = np.sqrt((coords[0] - indexBoundary[0])**2 + (coords[1] - indexBoundary[1])**2)

        if j==numTrials:
            trialDistances[0] = trialDistances.min(axis=0)
            j=0
        j+=1

    # find minimum distance to a boundary
    minDistances = trialDistances.min(axis=0)
    
    self.GBProxigram=minDistances

ebsd.Map.calcProxiGram=calcProxiGram

In [51]:
maps[0].calcProxiGram()

#### Plot proxigram map

In [203]:
def plotGBProxiGramMap(self):
    plt.figure()
    plt.imshow(self.GBProxigram*self.stepSize)
    plt.colorbar(label="Proximity to nearest GB [$\\mu$ m]",shrink=0.8)

ebsd.Map.plotGBProxiGramMap=plotGBProxiGramMap

In [204]:
maps[0].plotGBProxiGramMap()

#### Get proxigram for individual grain

In [55]:
def getGrainGBProxiGram(self,GBProxiGramMap=maps[0].GBProxigram):
    GBProxiGramList = []
    for coord in self.coordList:
        GBProxiGramVal = GBProxiGramMap[coord[1],coord[0]]
        GBProxiGramList.append(GBProxiGramVal)
    self.GBProxiGramList=GBProxiGramList

ebsd.Grain.getGrainGBProxiGram=getGrainGBProxiGram

In [56]:
for grain in maps[0].grainList:
    grain.getGrainGBProxiGram()

#### Plot grain proxigram

In [207]:
def plotGBProxigram(self):    
    plt.figure()
    x0, y0, xmax, ymax = self.extremeCoords
#     x0, y0, xmax, ymax = int(x0), int(y0), int(xmax), int(ymax)
    plotArea = np.full((ymax - y0 + 1, xmax - x0 + 1), np.nan, dtype=float)
    plotData = self.GBProxiGramList
    coord0s,coord1s=[],[]
    for coord0,coord1 in self.coordList:
        coord0s.append(coord0)
        coord1s.append(coord1)
    for coord0,coord1,data in zip(coord0s,coord1s,plotData):
        plotArea[coord1 - y0, coord0 - x0] = data
    plt.imshow(plotArea*maps[0].stepSize, cmap="viridis")
    plt.axis("off")
    plt.colorbar(label="Proximity to nearest GB [$\\mu$ m]",shrink=0.8)
    
ebsd.Grain.plotGBProxigram=plotGBProxigram

In [209]:
maps[0].grainList[DicMap.ebsdGrainIds[32]].plotGBProxigram()

#### Find misOri proxigram

In [213]:
def calcMisOriProxigram(self,plotData=False,plotMovingMean=False,plotLinearFit=False):
    
    # define the data
    grainmisOriArrayDeg = np.arccos(np.asarray(self.misOriList))*180/np.pi
    distances = np.asarray(self.GBProxiGramList)*maps[0].stepSize # um
    
    # bin the data
    from scipy.stats import binned_statistic
    bins = np.linspace(0,np.max(distances),int(np.max(distances))*2)
    
    if len(bins) > 3: # grain size must be sufficient to get a linear plot
    
        binnedDataMeans = binned_statistic(distances,grainmisOriArrayDeg,statistic='mean',bins=bins,range=(0,int(np.max(distances))))

        if plotData:
            plt.figure()
            plt.plot(distances,grainmisOriArrayDeg,".")
            plt.xlabel("Distance to GB [$\\mu$m]")
            plt.ylabel("EBSD pixel misorientation [ $^o$ ]")


        if plotData:
            if plotMovingMean:
                plt.plot(bins[:-1],binnedDataMeans[0][:])
        else:
            if plotMovingMean:
                plt.figure()
                plt.plot(bins[:-1],binnedDataMeans[0][:])
                plt.xlabel("Distance to GB [$\\mu$m]")
                plt.ylabel("EBSD pixel misorientation [ $^o$ ]")

        # model the curve with a linear function
        m,c = np.polyfit(bins[:-1],binnedDataMeans[0][:],1) # find linear fit to moving mean

        self.GBProxiGramGradient = m

        dataVals=[]
        for binVal in bins[:-1]:
            dataVal = binVal*m + c
            dataVals.append(dataVal)

        if plotData:
            if plotLinearFit:
                plt.plot(bins[:-1],dataVals)
        elif plotMovingMean:
            if plotLinearFit:
                plt.plot(bins[:-1],dataVals)
        else:
            if plotLinearFit:
                plt.figure()
                plt.plot(bins[:-1],dataVals)
                plt.xlabel("Distance to GB [$\\mu$m]")
                plt.ylabel("EBSD pixel misorientation [ $^o$ ]")
                
    else:
        self.GBProxiGramGradient = np.nan

ebsd.Grain.calcMisOriProxigram=calcMisOriProxigram

In [215]:
# check this works
maps[0].grainList[DicMap.ebsdGrainIds[32]].calcMisOriProxigram(plotData=True,plotMovingMean=True,plotLinearFit=True)
maps[0].grainList[DicMap.ebsdGrainIds[32]].GBProxiGramGradient

-0.0027179595228401169

#### Do for every grain

In [161]:
for grain in maps[0].grainList:
    grain.calcMisOriProxigram(plotData=False,plotMovingMean=False,plotLinearFit=False)

#### Plot distribution of misOri GB proxigram gradients

In [162]:
misOriGBProxigramGradients=[]
for grain in maps[0].grainList:
    misOriGBProxigramGradients.append(grain.GBProxiGramGradient)

In [174]:
# plot the histogram as a line plot
misOriGBProxigradGradient_hist = np.histogram(misOriGBProxigramGradients,range=(-0.2,0.2),bins=30)
x = 0.5*(misOriGBProxigradGradient_hist[1][1:]+misOriGBProxigradGradient_hist[1][:-1])
y = misOriGBProxigradGradient_hist[0]
plt.plot(x,y)
plt.xlabel("GB proxigram gradient, EBSD misorientation")
plt.ylabel("Grain number frequency")

/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/numpy/lib/function_base.py:747: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= mn)
/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/numpy/lib/function_base.py:748: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= mx)


## Deformation at boundaries relative to the grain centroid: HRDIC effective strain

#### Calcualte the proxigram (must calcualte EBSD proxigram first)

In [176]:
def calcDICProxiGram(self):
    minDistancesDic = DicMap.warpToDicFrame(maps[0].GBProxigram)
    DicMap.GBProxigram = minDistancesDic

hrdic.Map.calcDICProxiGram=calcDICProxiGram

In [177]:
DicMap.calcDICProxiGram()

#### Plot proxigram map

In [193]:
def plotGBProxiGramMap(self):
    plt.figure()
    plt.imshow(self.GBProxigram*maps[0].stepSize) # pixel distance values are warped from ebsd frame so need to use ebsd step size to charnge to units of um 
    plt.colorbar(label="Proximity to nearest GB [$\\mu$ m]",shrink=0.8)

hrdic.Map.plotGBProxiGramMap=plotGBProxiGramMap

In [195]:
DicMap.plotGBProxiGramMap()

#### Get proxigram for individual grain

In [196]:
def getGrainGBProxiGram(self,GBProxiGramMap=DicMap.GBProxigram):
    GBProxiGramList = []
    for coord in self.coordList:
        GBProxiGramVal = GBProxiGramMap[coord[1],coord[0]]
        GBProxiGramList.append(GBProxiGramVal)
    self.GBProxiGramList=GBProxiGramList

hrdic.Grain.getGrainGBProxiGram=getGrainGBProxiGram

In [197]:
for grain in DicMap.grainList:
    grain.getGrainGBProxiGram()

#### Plot grain proxigram

In [201]:
def plotGBProxigram(self):    
    plt.figure()
    x0, y0, xmax, ymax = self.extremeCoords
#     x0, y0, xmax, ymax = int(x0), int(y0), int(xmax), int(ymax)
    plotArea = np.full((ymax - y0 + 1, xmax - x0 + 1), np.nan, dtype=float)
    plotData = self.GBProxiGramList
    coord0s,coord1s=[],[]
    for coord0,coord1 in self.coordList:
        coord0s.append(coord0)
        coord1s.append(coord1)
    for coord0,coord1,data in zip(coord0s,coord1s,plotData):
        plotArea[coord1 - y0, coord0 - x0] = data
    plt.imshow(plotArea*maps[0].stepSize, cmap="viridis")
    plt.axis("off")
    plt.colorbar(label="Proximity to nearest GB [$\\mu$ m]",shrink=0.8)
    
hrdic.Grain.plotGBProxigram=plotGBProxigram

In [212]:
DicMap.grainList[32].plotGBProxigram()

#### Find misOri proxigram

In [218]:
def calcEffStrainProxigram(self,plotData=False,plotMovingMean=False,plotLinearFit=False):
    
    # define the data
    grainStrainData = self.maxShearList
    distances = np.asarray(self.GBProxiGramList)*maps[0].stepSize # um
    
    # bin the data
    from scipy.stats import binned_statistic
    bins = np.linspace(0,np.max(distances),int(np.max(distances))*2)
    
    if len(bins) > 3: # grain size must be sufficient to get a linear plot
    
        binnedDataMeans = binned_statistic(distances,grainStrainData,statistic='mean',bins=bins,range=(0,int(np.max(distances))))

        if plotData:
            plt.figure()
            plt.plot(distances,grainStrainData,".")
            plt.xlabel("Distance to GB [$\\mu$m]")
            plt.ylabel("HRDIC pixel strain, $\\epsilon_{eff}$ [  ]")


        if plotData:
            if plotMovingMean:
                plt.plot(bins[:-1],binnedDataMeans[0][:])
        else:
            if plotMovingMean:
                plt.figure()
                plt.plot(bins[:-1],binnedDataMeans[0][:])
                plt.xlabel("Distance to GB [$\\mu$m]")
                plt.ylabel("HRDIC pixel strain, $\\epsilon_{eff}$ [  ]")

        # model the curve with a linear function
        m,c = np.polyfit(bins[:-1],binnedDataMeans[0][:],1) # find linear fit to moving mean

        self.GBProxiGramGradient = m

        dataVals=[]
        for binVal in bins[:-1]:
            dataVal = binVal*m + c
            dataVals.append(dataVal)

        if plotData:
            if plotLinearFit:
                plt.plot(bins[:-1],dataVals)
        elif plotMovingMean:
            if plotLinearFit:
                plt.plot(bins[:-1],dataVals)
        else:
            if plotLinearFit:
                plt.figure()
                plt.plot(bins[:-1],dataVals)
                plt.xlabel("Distance to GB [$\\mu$m]")
                plt.ylabel("HRDIC pixel strain, $\\epsilon_{eff}$ [  ]")
                
    else:
        self.GBProxiGramGradient = np.nan

hrdic.Grain.calcEffStrainProxigram=calcEffStrainProxigram

In [220]:
# check this works
DicMap.grainList[32].calcEffStrainProxigram(plotData=True,plotMovingMean=True,plotLinearFit=True)
DicMap.grainList[32].GBProxiGramGradient

3.9233852861723832e-05

#### Do for every grain

In [221]:
for grain in DicMap.grainList:
    grain.calcEffStrainProxigram(plotData=False,plotMovingMean=False,plotLinearFit=False)

#### Plot distribution of misOri GB proxigram gradients

In [223]:
effStrainProxigramGradients=[]
for grain in DicMap.grainList:
    effStrainProxigramGradients.append(grain.GBProxiGramGradient)

In [224]:
# plot the histogram as a line plot
strainGBProxigradGradient_hist = np.histogram(effStrainProxigramGradients,range=(-0.2,0.2),bins=30)
x = 0.5*(strainGBProxigradGradient_hist[1][1:]+strainGBProxigradGradient_hist[1][:-1])
y = strainGBProxigradGradient_hist[0]
plt.plot(x,y)
plt.xlabel("GB proxigram gradient, HRDIC effective strain")
plt.ylabel("Grain number frequency")

/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/numpy/lib/function_base.py:747: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= mn)
/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/numpy/lib/function_base.py:748: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= mx)
